# EHT NETWORK

## Parâmetros


#### Frequency [5], frequency2 [0], frequency3 [0]:

Define as frequências usadas nos até 3 links possíveis usando Multi-Link Operation.
Os valores possíveis são: 2.4 ; 5 ; 6 ; 0. Por padrão, só está ativada uma frequência.


In [ ]:
channelStr[nLinks] = "{0, " + std::to_string(channelWidth) + ", ";
if (freq == 6)
{
    channelStr[nLinks] += "BAND_6GHZ, 0}";
    Config::SetDefault("ns3::LogDistancePropagationLossModel::ReferenceLoss",
                       DoubleValue(48));
    wifi.SetRemoteStationManager(nLinks,
                                 "ns3::ConstantRateWifiManager",
                                 "DataMode",
                                 StringValue(dataModeStr),
                                 "ControlMode",
                                 StringValue(dataModeStr));
}
else if (freq == 5)
{
    channelStr[nLinks] += "BAND_5GHZ, 0}";
    ctrlRateStr = "OfdmRate" + std::to_string(nonHtRefRateMbps) + "Mbps";
    wifi.SetRemoteStationManager(nLinks,
                                 "ns3::ConstantRateWifiManager",
                                 "DataMode",
                                 StringValue(dataModeStr),
                                 "ControlMode",
                                 StringValue(ctrlRateStr));
}
else if (freq == 2.4)
{
    channelStr[nLinks] += "BAND_2_4GHZ, 0}";
    Config::SetDefault("ns3::LogDistancePropagationLossModel::ReferenceLoss",
                       DoubleValue(40));
    ctrlRateStr = "ErpOfdmRate" + std::to_string(nonHtRefRateMbps) + "Mbps";
    wifi.SetRemoteStationManager(nLinks,
                                 "ns3::ConstantRateWifiManager",
                                 "DataMode",
                                 StringValue(dataModeStr),
                                 "ControlMode",
                                 StringValue(ctrlRateStr));
}

Este trecho configura os parametros de link para cada canal que tenha sido passado nos parametros de execução do script.

Se foi a frequencia foi escolhida, o algoritmo adiciona a banda ao _channelStr_ e define o _SetRemoteStationManager_.

O _SetRemoteStationManager_ é responável por definir os parametros passados para a estação no link definido pela variável _nLinks_.

In [ ]:
Ssid ssid = Ssid("ns3-80211be");

/*
 * SingleModelSpectrumChannel cannot be used with 802.11be because two
 * spectrum models are required: one with 78.125 kHz bands for HE PPDUs
 * and one with 312.5 kHz bands for, e.g., non-HT PPDUs (for more details,
 * see issue #408 (CLOSED))
 */
Ptr<MultiModelSpectrumChannel> spectrumChannel =
    CreateObject<MultiModelSpectrumChannel>();

Ptr<LogDistancePropagationLossModel> lossModel =
    CreateObject<LogDistancePropagationLossModel>();
spectrumChannel->AddPropagationLossModel(lossModel);

SpectrumWifiPhyHelper phy(nLinks);
phy.SetPcapDataLinkType(WifiPhyHelper::DLT_IEEE802_11_RADIO);
phy.SetChannel(spectrumChannel);

mac.SetType("ns3::StaWifiMac", "Ssid", SsidValue(ssid));
for (uint8_t linkId = 0; linkId < nLinks; linkId++)
{
    phy.Set(linkId, "ChannelSettings", StringValue(channelStr[linkId]));
}
staDevices = wifi.Install(phy, mac, wifiStaNodes);

Após a definição dos links que serão utilizados. É definido o ssid da rede;

Também é um MultiModelSpectrumChannel do tipo MultiModelSpectrumChannel.
O MultiModelSpectrumChannel é uma implementação do SpectrumChannel que pode lidar com diferentes modelos de espectro, ou seja, diferentes SpectrumModel e SpectrumPhy.

Depois é feita a criação de modelo de propagação e ele é atribuido ao spectrumChannel criado anteriormente.

Em seguida é criado o SpectrumWifiPhyHelper com o número de links q foi definido pelos parâmetros (nLinks).
Ao phy é definido um modelo de PCAP (método de captura de pacotes) e também o MultiModelSpectrumChannel criado anteriormente.

Por fim, ao WifiMacHelper (mac) é definido seu tipo e ssid e em seguida cada um dos links configurados anteriormene é atribuido ao SpectrumWifiPhyHelper (phy) as configurações de cada canal.

Feito isso, são instalados em cada device, seu respectivo nó, e configurações de phy e mac.

#### Distance [1]:

Define a distância em metros entre os STAs e o AP.
Por padrão, a distância é 1 metro.

In [ ]:
// mobility.
MobilityHelper mobility;
Ptr<ListPositionAllocator> positionAlloc = CreateObject<ListPositionAllocator>();

positionAlloc->Add(Vector(0.0, 0.0, 0.0));
positionAlloc->Add(Vector(distance, 0.0, 0.0));
mobility.SetPositionAllocator(positionAlloc);

mobility.SetMobilityModel("ns3::ConstantPositionMobilityModel");

mobility.Install(wifiApNode);
mobility.Install(wifiStaNodes);

A distância é definida nas configurações de mobilidade dos dispositivos.

#### SimulationTime [10]:

Define o tempo da simulação em segundos. Por padrão são 10s de simulação.

O tempo de simulação é utilizados para a definição de alguns parâmetros e calculos, como por exemplo, para calcular o Throughput.

In [ ]:
// UDP flow
uint16_t port = 9;
UdpServerHelper server(port);
serverApp = server.Install(serverNodes.get());
serverApp.Start(Seconds(0.0));
serverApp.Stop(Seconds(simulationTime + 1));

Na configuração de tráfego o tempo de simulação é usado para definir quando o servidor irá parar

In [ ]:
 double throughput = (rxBytes * 8) / (simulationTime * 1000000.0); // Mbit/s

Para Calcular o Throughput o tempo de simulação é usado para obtermos o resultado em Mbit/s uma vez que rxBytes é a quantidade de bytes acumulada ao longo da simulação.

#### UDP [True]:

Define o qual será o tipo de tráfego utilizado na simulação, se será UDP ou TCP. Por padrão é o UDP, esse valor é definido em uma variável booleana chamada udp, portanto:

- udp = true -> UPD
- udp = false -> TCP

Sabemos que o UDP e TCP são protocolos da camada de transporte e em resumo, o TCP é mais confiável, mas mais lento devido ao controle adicional. O UDP é mais rápido, mas não garante a entrega.

A depender do protocolo, algumas configurações são diferentes na montagem da simulação.

#### Download [True]:

Define o qual será o tipo de tráfego utilizado na simulação, se será UDP ou TCP. Por padrão é o UDP, esse valor é definido em uma variável booleana chamada udp, portanto:


In [ ]:
/* Setting applications */
ApplicationContainer serverApp;
auto serverNodes = downlink ? std::ref(wifiStaNodes) : std::ref(wifiApNode);
Ipv4InterfaceContainer serverInterfaces;
NodeContainer clientNodes;
for (std::size_t i = 0; i < nStations; i++)
{
    serverInterfaces.Add(downlink ? staNodeInterfaces.Get(i)
                                  : apNodeInterface.Get(0));
    clientNodes.Add(downlink ? wifiApNode.Get(0) : wifiStaNodes.Get(i));
}

aaaa